In [18]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision
from datasets import load_dataset
import matplotlib.pyplot as plt

In [19]:
# prepare dataset
class CatsDataset(Dataset):
    def __init__(self, train=True):
        if train:
            temp_dataset = load_dataset("cifar10")['train']
        else:
            temp_dataset = load_dataset("cifar10")['test']
        
        self.dataset = []
        for i in range(len(temp_dataset)):
            if temp_dataset[i]['label'] == 3:
                img = torchvision.transforms.PILToTensor()(temp_dataset[i]['img']).expand((3, -1, -1)) / 255.0
                self.dataset.append([img, torch.tensor([0])])

            elif torch.rand((1, )) < 1/9:
                img = torchvision.transforms.PILToTensor()(temp_dataset[i]['img']).expand((3, -1, -1)) / 255.0
                self.dataset.append([img, torch.tensor([1])])
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        image, label = self.dataset[idx]
        image = torchvision.transforms.RandomHorizontalFlip()(image)
        image = torchvision.transforms.RandomResizedCrop((32, 32), scale=(4/5, 5/4), ratio=(4/5, 5/4))(image)
        return (image, label)

In [20]:
train_dataset = CatsDataset(train=True)
len(train_dataset)

9947

In [21]:
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=256,
    shuffle=True
)

In [22]:
dev = torch.device("cpu")
if torch.cuda.is_available():
    dev = torch.device("cuda")
elif torch.backends.mps.is_available():
    dev = torch.device("mps")

print(f"Using device: {dev}")

Using device: mps


In [23]:
model = nn.Sequential(
    nn.Conv2d(3, 64, 3),
    nn.MaxPool2d((3, 3)),
    nn.LeakyReLU(),
    nn.Conv2d(64, 64, 3),
    nn.MaxPool2d((3, 3)),
    nn.LeakyReLU(),
    nn.Flatten(),
    nn.Linear(256, 64),
    nn.LeakyReLU(),
    nn.Linear(64, 2)
).to(dev)

In [38]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)

In [39]:
for epoch in range(20):
    s = 0
    for i, data in enumerate(train_dataloader):
        # Every data instance is an input + label pair
        inputs, labels = data
        inputs = inputs.to(dev)
        labels = labels.to(dev)
        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)
        labels = labels.view(-1)
        # Compute the loss and its gradients
        loss = criterion(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        s += loss.item()
    
    print(f"Epoch {epoch+1} Loss: {s/len(train_dataloader)}")

torch.save(model.to(dev).state_dict(), "model.pt")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Epoch 1 Loss: 0.2303597243168415
Epoch 2 Loss: 0.2216855757511579
Epoch 3 Loss: 0.22574728612716383
Epoch 4 Loss: 0.21387231579193702
Epoch 5 Loss: 0.2225229140275564
Epoch 6 Loss: 0.2211446838501172
Epoch 7 Loss: 0.21931532407418275
Epoch 8 Loss: 0.22609552511802086
Epoch 9 Loss: 0.22340567983113802
Epoch 10 Loss: 0.22543280476178879
Epoch 11 Loss: 0.2320138460550553
Epoch 12 Loss: 0.23016405143798926
Epoch 13 Loss: 0.2272138584118623
Epoch 14 Loss: 0.22688598892627618
Epoch 15 Loss: 0.21831772457330656
Epoch 16 Loss: 0.22534549121673292
Epoch 17 Loss: 0.2202605929894325
Epoch 18 Loss: 0.22407832397864416
Epoch 19 Loss: 0.22122520361191186
Epoch 20 Loss: 0.22534858645536962


In [25]:
test_dataset = CatsDataset(train=False)

In [26]:
test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=64,
    shuffle=True
)

In [40]:
good = 0
a = 0
for i, data in enumerate(test_dataloader):
    # Every data instance is an input + label pair
    inputs, labels = data
    inputs = inputs.to(dev)
    labels = labels.to(dev)
    outputs = model(inputs)
    labels = labels.view(-1)
    
    for i in range(len(outputs)):
        good += torch.argmax(outputs[i]) == labels[i]
        a += 1

print(good / a)

tensor(0.7975, device='mps:0')
